In [1]:
from pyquery import PyQuery as pq
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json
from pprint import PrettyPrinter as pp

In [17]:
with open('./candidate_pages.json') as data_file:    
    candidate_json = json.load(data_file)
    
candidate_json.copy()

In [18]:
for key,val in candidate_json.iteritems():
    if isinstance( val, int ):
        print key
    elif len(val) < 5:
        print key

In [69]:
def get_crimdata(year, page_text, url):
    crim_list = []
    crim_det_list = []
    success = 0
    crim_dict = {'year_election': year, 'url': url}
    header = ['serial_no','IPC_sections','Other']
    for h3 in pq(page_text)("h3"):
        if h3.text == "Details of Criminal Cases":
            tablecrims = pq(h3).next('table')
            for h3_2 in pq(tablecrims)("h3"):           
                if h3_2.text == "Cases where charges framed":
                    framed = pq(h3_2).next('table')
                    crim_dict['type'] = 'Framed'
                    for i,tr in enumerate(pq(framed)('tr')):
                        crim_dict_copy = crim_dict.copy()
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict_copy[header[j]] = child.text
                                    success = 1
                            if success == 1:
                                crim_list.append(crim_dict_copy)
                                success = 0
                    
                if h3_2.text == "Cases where Cognizance taken":
                    framed = pq(h3_2).next('table')
                    crim_dict['type'] = 'Cogniz'
                    for i,tr in enumerate(pq(framed)('tr')):
                        crim_dict_copy = crim_dict.copy()
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict_copy[header[j]] = child.text
                                    success = 1
                            if success == 1:
                                crim_list.append(crim_dict_copy)
                                success = 0
                                
                    
                if h3_2.text == "Cases where convicted":
                    framed = pq(h3_2).next('table')
                    crim_dict['type'] = 'Convict'
                    for i,tr in enumerate(pq(framed)('tr')):
                        crim_dict_copy = crim_dict.copy()
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict_copy[header[j]] = child.text
                                    success = 1
                            if success == 1:
                                crim_list.append(crim_dict_copy)
                                success = 0

        if h3.text == "Brief Details of IPCs":
            crim_det_dict = {'url': url}
            parent = pq(h3).parent()
            for x in pq(parent).children()[1:]:
                dets = pq(x).text()
                if len(dets) > 0:
                    crim_det_copy = crim_det_dict.copy()
                    crim_det_copy['detail'] = dets
                    crim_det_copy.append(crim_det_dict)
                
        
    return crim_det_list, crim_list



In [70]:
### FOR TESTING ###
test_index = "http://myneta.info/ls2014/candidate.php?candidate_id=3492"
page_data = candidate_json[test_index]
det_list, crim_list = get_crimdata(2014,page_data,test_index)

det_list
#for test_df

[{'detail': '1 charges related to Illegal payments in connection with an election (IPC Section-171H)',
  'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'},
 {'detail': '1 charges related to Illegal payments in connection with an election (IPC Section-171H)',
  'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'},
 {'detail': '1 charges related to Illegal payments in connection with an election (IPC Section-171H)',
  'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'}]

In [76]:
crimdataset = []

for i,urls in enumerate(candidate_json.keys()):
    page_data = candidate_json[urls]
    if (i%100) == 0:
        print ".",
    crimdataset = crimdataset + get_crimdata(2014,page_data,urls)[1]
    
criminal_df = pd.DataFrame(crimdataset)
criminal_df.count()

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

IPC_sections     4502
Other            4502
serial_no        4502
type             4502
url              4502
year_election    4502
dtype: int64

In [77]:
criminal_df.head()

,IPC_sections,Other,serial_no,type,url,year_election
0,"147, 148, 149, 307, 425","3rd ADJ,Sonbhadra,No-144/08,Date 12/07/2008",1,Cogniz,http://myneta.info/ls2014/candidate.php?candid...,2014
1,,"154 Cr.pc, Chief Judicical Magistrate Court K...",1,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2,"143, 147, 283, 149","JFCM IV Kozhikode ,CC 1217/2013(Nadakkavu Cr....",1,Cogniz,http://myneta.info/ls2014/candidate.php?candid...,2014
3,"143, 147, 149, 435, 288","Sec.4(1)r/w 6 of Kerala Public Ways Act, Sec....",2,Cogniz,http://myneta.info/ls2014/candidate.php?candid...,2014
4,,"ADDITIONAL CHIEF METO. COURT, DISA COURT. CRI...",1,Cogniz,http://myneta.info/ls2014/candidate.php?candid...,2014


In [78]:
criminal_df[criminal_df.url == 'http://myneta.info/ls2014/candidate.php?candidate_id=3492']

,IPC_sections,Other,serial_no,type,url,year_election
2204,"143, 147, 149, 323, 324, 336, 427","Judicial Magistrate Saheb, Class-1, R.T.C. 31...",1,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2205,"143, 149, 341","Judicial Magistrate Saheb, Class-1, R.T.C. 18...",2,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2206,"341, 143, 147","Judicial Magistrate Saheb, Class-1, R.T.C. 34...",3,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2207,"182, 171","Judicial Magistrate Saheb, Class-1, R.T.C. 22...",4,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2208,171H,"Judicial Magistrate Saheb, Class-1, R.T.C. 29...",5,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2209,,"Section 37(1)(3)/135 Mumbai Police Act, Judic...",6,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2210,"143, 147, 323, 504","Case No. 188/1987, Nagar Police Station\t",1,Cogniz,http://myneta.info/ls2014/candidate.php?candid...,2014
2211,"147, 148, 149, 323, 336, 504, 506","Case No.56/1985, Nagar Police Station",2,Cogniz,http://myneta.info/ls2014/candidate.php?candid...,2014
2212,"337, 323, 504, 506","Case No.145/1985, Nagar Police Station",3,Cogniz,http://myneta.info/ls2014/candidate.php?candid...,2014
2213,"143, 427","Case No.93/1990, Nagar Police Station",4,Cogniz,http://myneta.info/ls2014/candidate.php?candid...,2014


In [ ]:
criminal_df.to_csv('criminal_record.csv')